# Synthetically-Created, Human-Evaluated Reasoning Dataset

- CommonsenseQA, StrategyQA
- Varitions on questions / tasks / queries
- Human eval
- Think step-by-step reasoning prompts
- Huamn eval
- Answers to query + reasoning
- Human eval

Aim for 1,000 starting sampleset.


In [20]:
from datasets import load_dataset
from huggingface_hub import InferenceClient

from pprint import pprint
import random
import os

In [2]:
"""
CommonsenseQA from TAU (Tel Aviv University)

https://arxiv.org/abs/1811.00937

"""
dataset_id = 'tau/commonsense_qa'

dataset = load_dataset(dataset_id)
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'question_concept', 'choices', 'answerKey'],
        num_rows: 9741
    })
    validation: Dataset({
        features: ['id', 'question', 'question_concept', 'choices', 'answerKey'],
        num_rows: 1221
    })
    test: Dataset({
        features: ['id', 'question', 'question_concept', 'choices', 'answerKey'],
        num_rows: 1140
    })
})


In [3]:
train_dataset = dataset['train']
print(train_dataset)

Dataset({
    features: ['id', 'question', 'question_concept', 'choices', 'answerKey'],
    num_rows: 9741
})


In [44]:
len(train_dataset)

9741

In [41]:
q_no = 13
pprint(train_dataset[q_no])

{'answerKey': 'A',
 'choices': {'label': ['A', 'B', 'C', 'D', 'E'],
             'text': ['loss of heat',
                      'revenge',
                      'expansion',
                      'relaxation',
                      'calm down']},
 'id': 'b63b9809c203321d6659ddf8551894bf',
 'question': "James was cooling off two quickly.  He would die if he didn't "
             'find some way to stop what?',
 'question_concept': 'cooling off'}


In [43]:
import json
import time

model_id = 'mistralai/Mixtral-8x7B-Instruct-v0.1'

queries = random.sample(range(1, len(train_dataset) + 1), 1000)

responses = []

for q in queries:
    query = train_dataset[q]['question'].replace('  ', ' ')
    choices = train_dataset[q]['choices']['text']

    client = InferenceClient(
        model=model_id,
        token=os.getenv('HUGGINGFACE_TOKEN'),
    )

    prompt = f"Q: {query}\n\
        Choices: {', '.join(choices)}\n\
        A: Let's think step by step."
    pprint(prompt)

    seed = random.randint(0, 10000)

    while True:
        try:
            output = client.text_generation(
                prompt,
                max_new_tokens=50,
                do_sample=True,
                seed=seed
            )
            break
        except:
            print("An interruption occurred. Retrying in 3 minutes.")
            time.sleep(180)  
    
    response = {
        'id': train_dataset[q]['id'],
        'response': output['generated_text']
    }
    responses.append(response)
    time.sleep(random.uniform(1, 10))

# Save responses to a JSON file
with open('responses.json', 'w') as f:
    json.dump(responses, f)

("Q: James was cooling off two quickly. He would die if he didn't find some "
 'way to stop what?\n'
 '    Choices: loss of heat, revenge, expansion, relaxation, calm down\n'
 "    A: Let's think step by step.")
Response 1:  James is in aggressive surroundings and he's hot. He can regulate temperature if he's burning up, so the answer to our riddle is 'loss of heat'.
Response 2:  There is a story regarding James and some men trying to kill him. He's in his apartment and he knows they need just in to kill him. So he threw off his blanket during winter to increase his heart rate to produce more heat to stay
Response 3:  James set himself on fire "quickly," so he needs to find a way to stop the fire. The first thing which comes to mind here is loss of heat. A burning object takes in heat from its surroundings and that's why James was
